# Networkx ATLAS KG construction and RAG example
This notebook demonstrates the full streamlined process of creating a knowledge graph (KG) using the atlas-rag package and performing retrieval-augmented generation (RAG) with our created RAG methods.

## ATLAS KG construction
It is suggested to use local hf model to run the KG construction code, as llm api service provider use optimized, lightweight models to reduce costs, which may sacrifice performance, and hence hard to have guaranteed performance. (for example from fp16 to bf16 etc.)

ATLAS KG construction consist of 5 steps:
- Triples Json Generation (Base KG Json)
- Convert Triples Json to Triples csv
- Conceptualize Entity in Triples csv
- Merge Concept CSV to Triples CSV
- Convert CSV to graphml for networkx to perform rag

In [ ]:
from atlas_rag import TripleGenerator, KnowledgeGraphExtractor, ProcessingConfig
from openai import OpenAI
from transformers import pipeline
# client = OpenAI(api_key='<your_api_key>',base_url="<your_api_base_url>") 
# model_name = "meta-llama/llama-3.1-8b-instruct"

model_name = "meta-llama/Llama-3.1-8B-Instruct"
client = pipeline(
    "text-generation",
    model=model_name,
    device_map="auto",
)
keyword = 'Dulce'
output_directory = f'import/{keyword}'
triple_generator = TripleGenerator(client, model_name=model_name)

In [2]:
kg_extraction_config = ProcessingConfig(
      model_path=model_name,
      data_directory="tests",
      filename_pattern=keyword,
      batch_size=2,
      output_directory=f"{output_directory}",
)
kg_extractor = KnowledgeGraphExtractor(model=triple_generator, config=kg_extraction_config)

### Triples Generation (with OpenAI Package)

In [ ]:
# construct entity&event graph
kg_extractor.run_extraction()

In [ ]:
# Convert Triples Json to CSV
kg_extractor.convert_json_to_csv()

In [ ]:
# Concept Generation
kg_extractor.generate_concept_csv_temp(batch_size=64)

In [ ]:
kg_extractor.create_concept_csv()

## Choice 1: convert to graphml for networkx rag

In [7]:
# convert csv to graphml for networkx
kg_extractor.convert_to_graphml()

## Choice 2: Convert to neo4j dumps

In [ ]:
# add numeric id to the csv so that we can use vector indices
kg_extractor.add_numeric_id()

In [ ]:
kg_extractor.compute_embedding() # default encoder_model_name="all-MiniLM-L12-v2"
# kg_extractor.compute_embedding(encoder_model_name="all-MiniLM-L12-v2")
# kg_extractor.compute_embedding(encoder_model_name="nvidia/NV-Embed-v2")
kg_extractor.create_faiss_index() # default index_type="HNSW,Flat"

# kg_extractor.create_faiss_index(index_type="HNSW,Flat")
# kg_extractor.create_faiss_index(index_type="IVF65536_HNSW32,Flat")



## ATLAS RAG

In order to perform RAG, one need to first create embeddings & faiss index for constructed KG

[There maybe performance difference in using AutoModel and Sentence Transformer for NV-Ebmed-v2]

In [ ]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '1,2'  # Set to the GPU you want to use, or '0' for the first GPU
import torch
num_gpus = torch.cuda.device_count()
print("number of GPUs available:", torch.cuda.device_count())
for i in range(num_gpus):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

In [ ]:
from sentence_transformers import SentenceTransformer
from atlas_rag.retriever import NvEmbed
from transformers import AutoModel
# Load the SentenceTransformer model
encoder_model_name = "nvidia/NV-Embed-v2"
# sentence_model = SentenceTransformer(encoder_model_name, trust_remote_code=True, model_kwargs={'device_map': "auto"})
# sentence_model.max_seq_length = 32768
# sentence_model.tokenizer.padding_side="right"
sentence_model = AutoModel.from_pretrained(encoder_model_name, trust_remote_code=True, device_map="auto")
sentence_encoder = NvEmbed(sentence_model)

In [3]:
from openai import OpenAI
from atlas_rag.reader import LLMGenerator
from configparser import ConfigParser
# Load OpenRouter API key from config file
config = ConfigParser()
config.read('config.ini')
# reader_model_name = "meta-llama/llama-3.3-70b-instruct"
reader_model_name = "meta-llama/Llama-3.3-70B-Instruct"
client = OpenAI(
  # base_url="https://openrouter.ai/api/v1",
  # api_key=config['settings']['OPENROUTER_API_KEY'],
  base_url="https://api.deepinfra.com/v1/openai",
  api_key=config['settings']['DEEPINFRA_API_KEY'],
)
llm_generator = LLMGenerator(client=client, model_name=reader_model_name)

In [ ]:
from atlas_rag import create_embeddings_and_index
keyword = 'musique'
working_directory = f'/data/httsangaj/atomic-rag/8b'
data = create_embeddings_and_index(
    sentence_encoder=sentence_encoder,
    model_name = 'nvidia/NV-Embed-v2',
    working_directory=working_directory,
    keyword=keyword,
    include_concept=True,
    include_events=True,
    normalize_embeddings= True,
    batch_size=32,
)

In [ ]:
from atlas_rag.evaluation import BenchMarkConfig
benchmark_config = BenchMarkConfig(
    dataset_name= 'musique',
    question_file= "benchmark_data/musique.json",
    include_concept=True,
    include_events=True,
    reader_model_name=reader_model_name,
    encoder_model_name=encoder_model_name,
    number_of_samples=-1, # -1 for all samples
)

In [ ]:
from atlas_rag import setup_logger
logger = setup_logger(benchmark_config)

In [ ]:
# Initialize desired RAG method for benchmarking
from atlas_rag.retriever import HippoRAG2Retriever
hipporag2_retriever = HippoRAG2Retriever(
    llm_generator=llm_generator,
    sentence_encoder=sentence_encoder,
    data = data,
    logger=logger
)

## Investigation for reason to perfomance difference:
- Version difference for cuda?
- Version difference for huggingface?

In [ ]:
# start benchmarking
from atlas_rag.evaluation import RAGBenchmark
benchmark = RAGBenchmark(config=benchmark_config, logger=logger)
benchmark.run([hipporag2_retriever], llm_generator=llm_generator)

## Billion Level KG RAG

from atlas_rag.billion import 